In [1]:
!mkdir -p ~/.config/Ultralytics/
!wget http://mirror.coggle.club/yolo/Arial.ttf -O ~/.config/Ultralytics/Arial.ttf
!wget http://mirror.coggle.club/yolo/Arial.Unicode.ttf -O ~/.config/Ultralytics/Arial.Unicode.ttf
!wget http://mirror.coggle.club/yolo/yolov8n-v8.2.0.pt -O yolov8n.pt
# !wget http://mirror.coggle.club/yolo/yolov8s-v8.2.0.pt -O yolov8s.pt
!wget http://mirror.coggle.club/yolo/yolov8m-v8.2.0.pt -O yolov8m.pt

--2025-07-12 16:42:35--  http://mirror.coggle.club/yolo/Arial.ttf
正在解析主机 mirror.coggle.club (mirror.coggle.club)... 222.134.1.30, 222.134.1.28, 222.134.1.29
正在连接 mirror.coggle.club (mirror.coggle.club)|222.134.1.30|:80... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 773236 (755K) [font/ttf]
正在保存至: ‘/home/lyz/.config/Ultralytics/Arial.ttf’

/home/lyz/.config/U 100%[===================>] 755.11K  3.95MB/s    用时 0.2s    

2025-07-12 16:42:35 (3.95 MB/s) - 已保存 ‘/home/lyz/.config/Ultralytics/Arial.ttf’ [773236/773236])

--2025-07-12 16:42:36--  http://mirror.coggle.club/yolo/Arial.Unicode.ttf
正在解析主机 mirror.coggle.club (mirror.coggle.club)... 222.134.1.28, 222.134.1.29, 222.134.1.30
正在连接 mirror.coggle.club (mirror.coggle.club)|222.134.1.28|:80... 已连接。
已发出 HTTP 请求，正在等待回应... 200 OK
长度： 23278008 (22M) [application/x-font-ttf]
正在保存至: ‘/home/lyz/.config/Ultralytics/Arial.Unicode.ttf’

/home/lyz/.config/U 100%[===================>]  22.20M  8.12MB/s    用时 2.7s    

2025-07-12 16:42:39 (8.12 MB/s) - 已保存 ‘

In [16]:
!\rm -rf train_yolo

In [17]:
import os
import glob
import shutil

import os
import xml.etree.ElementTree as ET
from glob import glob

def convert_voc_to_yolo(xml_path, output_dir, class_names):
    """
    Convert Pascal VOC XML annotation to YOLO format
    
    Args:
        xml_path (str): Path to XML file
        output_dir (str): Directory to save YOLO txt files
        class_names (list): List of class names in order
    """
    # Parse XML
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    # Get image dimensions
    size = root.find('size')
    img_width = int(size.find('width').text)
    img_height = int(size.find('height').text)
    
    # Create output file path
    image_name = os.path.splitext(os.path.basename(xml_path))[0]
    txt_path = os.path.join(output_dir, f"{image_name}.txt")
    
    with open(txt_path, 'w') as f:
        for obj in root.iter('object'):
            # Get class name
            cls_name = obj.find('name').text
            if cls_name not in class_names:
                continue  # skip classes not in our list
            cls_id = class_names.index(cls_name)
            
            # Get bounding box coordinates
            bndbox = obj.find('bndbox')
            xmin = int(bndbox.find('xmin').text)
            ymin = int(bndbox.find('ymin').text)
            xmax = int(bndbox.find('xmax').text)
            ymax = int(bndbox.find('ymax').text)
            
            # Convert to YOLO format (center x, center y, width, height) - normalized
            x_center = ((xmin + xmax) / 2) / img_width
            y_center = ((ymin + ymax) / 2) / img_height
            width = (xmax - xmin) / img_width
            height = (ymax - ymin) / img_height
            
            # Write to file
            f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

def main():
    # Configuration
    img_dir = "train"
    xml_dir = "train_label"  # Directory containing XML files
    img_dir = "train"
    output_dir = "train_yolo/train/labels"  # Directory to save YOLO format labels
    img_output_dir = "train_yolo/train/images"  # Directory to save YOLO format labels
    class_names = ["car", "truck", "bike", "human"]  # List your class names here, e.g., ["person", "car", "dog"]
    
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    os.makedirs(img_output_dir, exist_ok=True)

    # Get all XML files
    xml_files = glob(os.path.join(xml_dir, "*.xml"))
    
    img_files = glob(os.path.join(img_dir, "*.jpg"))
    for path in img_files:
        shutil.copy(path, img_output_dir)
    
    # If class_names is empty, collect all unique class names from XML files
    if not class_names:
        unique_classes = set()
        for xml_file in xml_files:
            tree = ET.parse(xml_file)
            root = tree.getroot()
            for obj in root.iter('object'):
                cls_name = obj.find('name').text
                unique_classes.add(cls_name)
        class_names = sorted(list(unique_classes))
        print(f"Found classes: {class_names}")
    
    # Process each XML file
    for xml_file in xml_files:
        convert_voc_to_yolo(xml_file, output_dir, class_names)
    
    print(f"Conversion complete. YOLO format labels saved to {output_dir}")
    
main()

Conversion complete. YOLO format labels saved to train_yolo/train/labels


In [18]:
os.makedirs('train_yolo/val', exist_ok=True)
os.makedirs('train_yolo/val/labels', exist_ok=True)
os.makedirs('train_yolo/val/images', exist_ok=True)

labels = glob('train_yolo/train/labels/*')
images = glob('train_yolo/train/images/*')

labels.sort(); images.sort()

for x, y in zip(labels[-50:], images[-50:]):
    shutil.move(x, 'train_yolo/val/labels/')
    shutil.move(y, 'train_yolo/val/images/')

In [1]:
import os
dir_path = os.path.abspath('./') + '/'

# 需要按照你的修改path
with open('yolo.yaml', 'w', encoding='utf-8') as up:
    up.write(f'''
path: {dir_path}/
train: train_yolo/train/
val: train_yolo/val/

names:
    0: car
    1: truck
    2: bike
    3: human
''')

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import warnings
warnings.filterwarnings('ignore')


from ultralytics import YOLO
model = YOLO("yolov8m.pt")
results = model.train(data="./yolo.yaml", epochs=20, device=1, batch=4, imgsz=1280)

New https://pypi.org/project/ultralytics/8.3.165 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.2.81 🚀 Python-3.11.8 torch-2.4.0+cu121 CUDA:1 (NVIDIA GeForce GTX 1080 Ti, 11163MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=./yolo.yaml, epochs=20, time=None, patience=100, batch=4, imgsz=1280, save=True, save_period=-1, cache=False, device=1, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_f

train: Scanning /home/lyz/Work/Kaggle/科大讯飞2025/车载摄像机图像的目标检测挑战赛 赛季2/train_yolo/train/labels.cache... 350 images, 0 backgrounds, 0 corrupt: 100%|██████████| 350/350 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/lyz/Work/Kaggle/科大讯飞2025/车载摄像机图像的目标检测挑战赛 赛季2/train_yolo/val/labels.cache... 50 images, 0 backgrounds, 0 corrupt: 100%|██████████| 50/50 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.00125, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 1280 train, 1280 val
Using 8 dataloader workers
Logging results to runs/detect/train4
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

      20/20      9.54G      1.564     0.8463      1.343         26       1280: 100%|██████████| 88/88 [01:10<00:00,  1.25it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.28it/s]

                   all         50        272      0.525      0.411      0.413      0.181



20 epochs completed in 0.406 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 52.1MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 52.1MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.2.81 🚀 Python-3.11.8 torch-2.4.0+cu121 CUDA:1 (NVIDIA GeForce GTX 1080 Ti, 11163MiB)
Model summary (fused): 218 layers, 25,842,076 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  4.14it/s]


                   all         50        272      0.704      0.358      0.468      0.202
                   car         50        214      0.875      0.537      0.678       0.34
                 truck          9         10      0.643        0.4      0.431      0.197
                  bike         22         44          1      0.246      0.469      0.176
                 human          4          4      0.297       0.25      0.295     0.0944
Speed: 0.4ms preprocess, 29.2ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs/detect/train4


In [3]:
model = YOLO("runs/detect/train4/weights/best.pt")

In [13]:
from glob import glob
import os

os.makedirs('submit', exist_ok=True)
test_paths = glob('test/*')

class_names = ["car", "truck", "bike", "human"]

for path in test_paths:
    predictions = model(path, save_txt=None)  # yolo model
    
    # Get the base filename without extension
    base_name = os.path.splitext(os.path.basename(path))[0]
    txt_path = os.path.join('submit', f"{base_name}.txt")
    
    with open(txt_path, 'w') as f:
        for idx in range(len(predictions[0].boxes.cls)):
            # Get class name
            cls_id = int(predictions[0].boxes.cls[idx].item())
            class_name = class_names[cls_id]
            
            # Get confidence
            confidence = predictions[0].boxes.conf[idx].item()
            
            # Get unnormalized bounding box coordinates (xyxy format)
            x1, y1, x2, y2 = predictions[0].boxes.xyxy[idx].tolist()
            
            # Write to file in specified format
            line = f"{class_name} {confidence:.6f} {int(x1)} {int(y1)} {int(x2)} {int(y2)}\n"
            f.write(line)


image 1/1 /home/lyz/Work/Kaggle/科大讯飞2025/车载摄像机图像的目标检测挑战赛 赛季2/test/1616442501812.jpg: 704x1280 7 cars, 24.9ms
Speed: 21.2ms preprocess, 24.9ms inference, 2.1ms postprocess per image at shape (1, 3, 704, 1280)

image 1/1 /home/lyz/Work/Kaggle/科大讯飞2025/车载摄像机图像的目标检测挑战赛 赛季2/test/1616535188128.jpg: 704x1280 9 cars, 24.9ms
Speed: 7.5ms preprocess, 24.9ms inference, 1.3ms postprocess per image at shape (1, 3, 704, 1280)

image 1/1 /home/lyz/Work/Kaggle/科大讯飞2025/车载摄像机图像的目标检测挑战赛 赛季2/test/1616535166200.jpg: 704x1280 1 car, 25.0ms
Speed: 6.1ms preprocess, 25.0ms inference, 1.1ms postprocess per image at shape (1, 3, 704, 1280)

image 1/1 /home/lyz/Work/Kaggle/科大讯飞2025/车载摄像机图像的目标检测挑战赛 赛季2/test/1618706253499.jpg: 704x1280 4 cars, 22.5ms
Speed: 5.2ms preprocess, 22.5ms inference, 1.1ms postprocess per image at shape (1, 3, 704, 1280)

image 1/1 /home/lyz/Work/Kaggle/科大讯飞2025/车载摄像机图像的目标检测挑战赛 赛季2/test/1616344839899.jpg: 704x1280 12 cars, 2 bikes, 2 humans, 22.5ms
Speed: 5.0ms preprocess, 22.5ms infere

In [15]:
!zip -r submit.zip submit

  adding: submit/ (stored 0%)
  adding: submit/1616534851699.txt (deflated 53%)
  adding: submit/1616535070700.txt (deflated 56%)
  adding: submit/1616534907199.txt (deflated 43%)
  adding: submit/1616442919800.txt (deflated 52%)
  adding: submit/1616535175111.txt (stored 0%)
  adding: submit/1616535103722.txt (deflated 55%)
  adding: submit/1616349559800.txt (deflated 39%)
  adding: submit/1616535144200.txt (deflated 52%)
  adding: submit/1616451500799.txt (deflated 55%)
  adding: submit/1616344127700.txt (deflated 51%)
  adding: submit/1616344181700.txt (deflated 53%)
  adding: submit/1616451393299.txt (deflated 31%)
  adding: submit/1616535234199.txt (deflated 46%)
  adding: submit/1616451659299.txt (deflated 36%)
  adding: submit/1618714256600.txt (deflated 25%)
  adding: submit/1616175997800.txt (deflated 39%)
  adding: submit/1616343614201.txt (deflated 42%)
  adding: submit/1616348509810.txt (deflated 34%)
  adding: submit/1618702725399.txt (deflated 52%)
  adding: submit/161645